In [35]:
!pip install facebook_business

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
!pip install pygsheets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
#Dùng để get dữ liệu từ Facebook API
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adcreative import AdCreative
import requests

#Dùng để get thời gian
from datetime import datetime, timedelta
import time

#Dùng để work với gg sheet
import pygsheets

#Một số thư viện khác
import pandas as pd

In [38]:
#Mở gg sheet
client = pygsheets.authorize(service_file = '/content/refined-grammar-372209-b3fbc4b10ebf.json')

sh = client.open('API BOT Tracking')
wks = sh.worksheet_by_title('AdAccounts')

#Lấy dữ liệu từ CampaignPeriod và lưu thành DataFrame
df = wks.get_as_df()

#Loại bỏ những dòng có access_token là null
drop_rows = df[df['access_token']==''].index
df.drop(drop_rows,axis = 0,inplace = True)

In [39]:
def df_from_insights(insights,access_token):
  data = pd.DataFrame(columns = ['ad_id','date','account_name','campaign_name','adset_name','reach','impressions','cost','post_engagement','message_ads','page_like_ads','post_body','post_url','post_image'])

  #Tạo ra các list rỗng để append giá trị
  date = []
  campaign_name = []
  adset_name = []
  reach = []
  impressions = []
  cost = []
  post_engagement = []
  mess_ads = []
  page_likes_ads = []
  ad_id = []
  account_name = []
  #Lặp qua tất cả các giá trị lấy về được từ get_insights(), nó bao gồm nhiều dict, mỗi dict là một dòng dữ liệu
  for i in insights:
    campaign_name.append(i['campaign_name'])
    adset_name.append(i['adset_name'])
    ad_id.append(i['ad_id'])
    date.append(i['date_start'])
    account_name.append(i['account_name'])
    reach.append(i['reach'])
    impressions.append(i['impressions'])
    cost.append(i['spend'])
    #Lấy ra post_engagement của bài ads, nếu không có => trả về 00
    try:
      value = ""
      for k in i['actions']:
        if k['action_type'] == 'post_engagement':
          value = k['value']
      post_engagement.append(value)
    except KeyError:
      post_engagement.append("")

    #Lấy ra message_ads của bài ads, nếu không có => trả về 00
    try:
      value = ""
      for k in i['actions']:
        if k['action_type'] == 'onsite_conversion.messaging_conversation_started_7d':
          value = k['value']
      mess_ads.append(value)
    except KeyError:
      mess_ads.append("")

    #Lấy ra page_likes_ads của bài ads, nếu không có => trả về 00
    try:
      value = ""
      for k in i['actions']:
        if k['action_type'] == 'likes':
          value = k['value']
      page_likes_ads.append(value)
    except KeyError:
      page_likes_ads.append("")

  #Gom tất cả lại thành 1 DataFrame
  df = pd.DataFrame({"campaign_name":campaign_name,
                      "adset_name":adset_name,
                      "account_name": account_name,
                      "ad_id":ad_id,
                      "date":date,
                      "reach":reach,
                      "impressions":impressions,
                      "cost":cost,
                      "post_engagement":post_engagement,
                      "message_ads":mess_ads,
                      "page_like_ads":page_likes_ads}, 
                      columns = ['ad_id','date','account_name','campaign_name','adset_name','reach','impressions','cost','post_engagement','message_ads','page_like_ads'])

  #Tạo list rỗng để append các giá trị
  ad_body = []
  post_url = []
  image_url = []

  #Lấy về các thông tin bằng cách requests graph api của Facebook, tham khảo thêm tại https://developers.facebook.com/docs/graph-api/
  for i in df['ad_id']:
    query = '''https://graph.facebook.com/v15.0/{ad_id}?fields=adcreatives{body,effective_object_story_id,thumbnail_url}
              &access_token={access_token}'''

    run_query = query.replace("{ad_id}",str(i)).replace('{access_token}',str(access_token))
    r = requests.get(run_query)     #Thay id của từng bài post vào để requests
    adcreatives = r.json()          #Thêm .json() để trả về các chuỗi có cả emoji
    #Lấy ra ad_body bằng cách index vào dictionary và thêm vào list 
    try:
      ad_body.append(adcreatives['adcreatives']['data'][0]["body"])
    except KeyError:
      ad_body.append("")

    #Lấy ra post_url bằng cách index vào dictionary và thêm vào list, post_url =  www.facebook.com/{effective_object_story_id}
    try:
      post_url.append("www.facebook.com/"+adcreatives['adcreatives']['data'][0]["effective_object_story_id"])
    except KeyError:
      post_url.append("")

    #Lấy ra image_url bằng cách index vào dictionary và thêm vào list, vì chưa lấy được image nên dùng thumbnail_url để thay thế 
    try:
      image_url.append(adcreatives['adcreatives']['data'][0]["thumbnail_url"])
    except KeyError:
      image_url.append("")
  

  #Nối list thành 3 cột trong DataFrame
  df['post_body'] = ad_body
  df['post_url'] = post_url
  df['post_image'] = image_url
  data = pd.concat([data,df], axis =0)
  return data

In [43]:
#requests 1 lần để lấy về ngày đầu, cuối tài khoản FB này chạy ads
def generate_period(id):

  #Các trường muốn trả về
  fields = ['date_start','date_stop']

  #Lấy insights thông qua AdAccount và Facebook API đã gọi từ trước
  date_insight = AdAccount(id).get_insights(fields)
  date_start = date_insight[0]['date_start']
  date_stop = date_insight[0]['date_stop']

  #Tạo list rỗng để chứa các giá trị
  date_lst = []

  #Gọi ra ngày hôm nay và đưa về đúng format thường sử dụng
  today =  datetime.strptime(datetime.today().strftime('%Y-%m-%d'),'%Y-%m-%d')

  #Ngày bắt đầu lấy dữ liệu mặc định là ngày 2022-10-01, tuy nhiên để requests ít hơn sẽ tìm ngày bắt đầu chạy của account FB và requests, nếu phát hiện accounts nào có chạy trước 2022-10-01 thì sẽ raise lên, muốn chạy trước đó thì sửa lại code
  if datetime.strptime(date_start,'%Y-%m-%d') >= datetime.strptime('2022-10-01','%Y-%m-%d'):                  #Nếu date_start lớn hơn 2022-10-01 thì lấy date_start làm ngày bắt đầu
    date_lst.append(date_start)

    if datetime.strptime(date_stop, '%Y-%m-%d') + timedelta(1) == today:                                      #Nếu ngày kết thúc + 1 = ngày hôm nay thì vẫn tiếp tục lấy từ date_start - today
      while datetime.strptime(date_lst[-1], '%Y-%m-%d') < today:
        date_lst.append((datetime.strptime(date_lst[-1], '%Y-%m-%d') + timedelta(1)).strftime('%Y-%m-%d'))
    
    else:
      while datetime.strptime(date_lst[-1], '%Y-%m-%d') < datetime.strptime(date_stop, '%Y-%m-%d'):
        date_lst.append((datetime.strptime(date_lst[-1], '%Y-%m-%d') + timedelta(1)).strftime('%Y-%m-%d'))

  else:                                                                                                       #Nếu date_start bé hơn 2022-10-01 thì lấy 2022-10-01 làm ngày bắt đầu
    print(f'Account {id} có campaign chạy trước ngày 2022-10-01, nếu muốn in ra thì chỉnh lại code')
    date_lst.append('2022-10-01')

    if datetime.strptime(date_stop, '%Y-%m-%d') + timedelta(1) == today:
      while datetime.strptime(date_lst[-1], '%Y-%m-%d') < today:
        date_lst.append((datetime.strptime(date_lst[-1], '%Y-%m-%d') + timedelta(1)).strftime('%Y-%m-%d'))
    
    else:
      while datetime.strptime(date_lst[-1], '%Y-%m-%d') < datetime.strptime(date_stop, '%Y-%m-%d'):
        date_lst.append((datetime.strptime(date_lst[-1], '%Y-%m-%d') + timedelta(1)).strftime('%Y-%m-%d'))

  return date_lst

In [44]:
def get_data(id, app_id, app_secret,access_token):

  output = pd.DataFrame(columns = ['date','account_name','campaign_name','adset_name','reach','impressions','cost','post_engagement','message_ads','page_like_ads','post_body','post_url','post_image'])


  #Gọi Facebook API
  FacebookAdsApi.init(app_id, app_secret, access_token)
  print("init")
  
  #Khởi tạo danh sách ngày chạy ads để requests 
  date_lst = generate_period(id)

  #Các trường mong muốn trả về
  fields = [
  'account_name',             #Tên tài khoản
  'campaign_name',            #Tên campaign
  'adset_name',               #Tên adset
  'ad_id',                    #Mã bài ad => Dùng để lấy về body, url, image_url
  'actions',                  #Tương tác bài ads, có nhiều trường nhưng chỉ sử dụng post_engagement, onsite_conversion.messaging_conversation_started_7d, likes
  'reach',                    #Reach
  'impressions',              #Impressions
  'spend']                    #Cost
    
  #Các parameters truyền vào
  params = {
      'level': 'ad',              #lấy dữ liệu theo level: có 4 level là adaccount, campaign, adset, ad. Cái này đang lấy theo level ad
      'limit':10000,              #giới hạn số lượng kết quả trả về
      'time_range[since]':'',     #Ngày bắt đầu
      'time_range[until]':'',     #Ngày kết thúc
  }

  #Lặp qua từng ngày trong date_lst để lấy dữ liệu theo daily
  if len(date_lst) >0:
    print(f"Period: {date_lst[0]} - {date_lst[-1]}")
    for i in date_lst:
      print('Lấy data ngày:',i)

      #Thay ngày vào params
      params['time_range[since]'] = i
      params['time_range[until]'] = i

      #Lấy insights
      insights = AdAccount(id).get_insights(
          params = params,
          fields = fields
      )
      print('Lượng Data:',len(insights))

      #Nếu insights > 0 thì lấy dữ liệu và lưu thành DataFrame
      if len(insights) >0:
        data_daily = df_from_insights(insights,access_token)
        print('Đã lấy về được:',len(data_daily))
        output = pd.concat([output,data_daily],axis=0)

  #Sửa đổi thứ tự cột trước khi return
  return output[['date','account_name','campaign_name','adset_name','post_body','post_url','post_image','reach','impressions','cost','post_engagement','message_ads','page_like_ads']]

In [45]:
while True:
  print("Bắt đầu lấy dữ liệu FB Ads")
  output = pd.DataFrame(columns = ['date','account_name','campaign_name','adset_name','post_body','post_url','post_image','reach','impressions','cost','post_engagement','message_ads','page_like_ads'])
  
  #Lặp qua các account trong DataFrame để lấy dữ liệu
  for i in range(0,len(df)):
    print('Đang lấy data từ:',df.iloc[i]['account_name'])

    #Gọi hàm get_data để lấy dữ liệu
    adaccount_data = get_data(df.iloc[i]['account_id'],df.iloc[i]['app_id'],df.iloc[i]['app_secret'],df.iloc[i]['access_token'])

    #Nối dữ liệu vừa tìm được vào output
    output = pd.concat([output,adaccount_data], axis =0)
    print('Xong account:', df.iloc[i]['account_name'], '- Lượng Data:', len(adaccount_data))
    print('---------------------')
    print('')

  #Ghi vào gg sheet
  write_sheet = sh.worksheet_by_title('FB Ads BOT')
  write_sheet.set_dataframe(output,start = "D2",copy_head =False)

  #Xong rồi đó, nghỉ 1 tiếng rồi chạy lại
  print('------DONE------')
  print("Nghỉ 1 tiếng rồi chạy tiếp")
  print('')
  time.sleep(3600)

Bắt đầu lấy dữ liệu FB Ads
Đang lấy data từ: Maido Agency Co., LTD
init
Period: 2022-12-17 - 2023-01-16
Lấy data ngày: 2022-12-17
Lượng Data: 0
Lấy data ngày: 2022-12-18
Lượng Data: 0
Lấy data ngày: 2022-12-19
Lượng Data: 0
Lấy data ngày: 2022-12-20
Lượng Data: 0
Lấy data ngày: 2022-12-21
Lượng Data: 0
Lấy data ngày: 2022-12-22
Lượng Data: 0
Lấy data ngày: 2022-12-23
Lượng Data: 0
Lấy data ngày: 2022-12-24
Lượng Data: 0
Lấy data ngày: 2022-12-25
Lượng Data: 0
Lấy data ngày: 2022-12-26
Lượng Data: 0
Lấy data ngày: 2022-12-27
Lượng Data: 0
Lấy data ngày: 2022-12-28
Lượng Data: 0
Lấy data ngày: 2022-12-29
Lượng Data: 0
Lấy data ngày: 2022-12-30
Lượng Data: 0
Lấy data ngày: 2022-12-31
Lượng Data: 0
Lấy data ngày: 2023-01-01
Lượng Data: 0
Lấy data ngày: 2023-01-02
Lượng Data: 0
Lấy data ngày: 2023-01-03
Lượng Data: 0
Lấy data ngày: 2023-01-04
Lượng Data: 0
Lấy data ngày: 2023-01-05
Lượng Data: 0
Lấy data ngày: 2023-01-06
Lượng Data: 0
Lấy data ngày: 2023-01-07
Lượng Data: 32
Đã lấy về được:

Lượng Data: 0
Xong account: Khoi Nguyen - Lượng Data: 2186
